In [1]:
from bs4 import BeautifulSoup
# from smtplib import smtplib
import requests
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import gspread
import pygsheets
import sqlite3
import smtplib

ImportError: No module named oauth2client.service_account

In [ ]:
# open jail log and use data to create Pandas dataframe
url = "http://oldweb.circuit8.org/inmatelist.php"
html = urlopen(url)
soup = BeautifulSoup(html, "lxml")
data_rows = soup.findAll('tr')[1:]
inmate_name = [[td.getText() for td in data_rows[i].findAll('td')]
              for i in range(len(data_rows))]
column_headers = ['inmate_name', 'pod', 'booking_date', 'MNInum', 'image_link']
jail_df = pd.DataFrame(inmate_name, columns=column_headers)
jail_df = jail_df.applymap(lambda x: str(x).strip() if len(str(x).strip()) else None) # get rid of extra white space
# separate joined first-middle-last name into fields for first, middle and last; deal with those with only first-last
# no solution here for with inmates with four or more names
jail_df.loc[jail_df['inmate_name'].str.split().str.len() == 3, 'first_name'] = jail_df['inmate_name'].str.split().str[-2]
jail_df.loc[jail_df['inmate_name'].str.split().str.len() == 3, 'middle_name'] = jail_df['inmate_name'].str.split().str[-1]
jail_df.loc[jail_df['inmate_name'].str.split().str.len() == 3, 'last_name'] = jail_df['inmate_name'].str.split().str[-0]
jail_df['last_name'].fillna(jail_df['inmate_name'], inplace=True)
jail_df['first_name'].fillna(jail_df['inmate_name'], inplace=True)
jail_df['first_name'] = jail_df['first_name'].str.replace(r'^[A-Z].+\s', '')
jail_df['last_name'] = jail_df['last_name'].str.replace(r',\s[A-Z]*', '') 
jail_df['last_name'] = jail_df['last_name'].apply(lambda x:str(x).replace(',','')) 
jail_df['firstlast'] = jail_df['first_name'].astype(str) + jail_df['last_name'] #create 'firstlast' col for key

In [ ]:
# access Google sheet with VIP names to compare to jail log
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
gc = pygsheets.authorize(service_file='AlachuaJailAlerts-8f6b7cd78fd0.json')
vip_df = pd.DataFrame()
sheet = gc.open('vip_names').sheet1
vip_df = pd.DataFrame(sheet.get_all_records())
jailed_vip = jail_df[jail_df.firstlast.isin(vip_df.firstlast)] #filter

In [ ]:
# send out email alert
if len(jailed_vip > 0):
    sent_from = 'ocagnv@gmail.com'  
    to = ['doug.ray@starbanner.com']  
    subject = 'OMG Super Important Message'  
    body = 'Hey, what up?\n\n- You'

    email_text = """\  
    From: %s  
    To: %s  
    Subject: %s

    %s
    """ % (sent_from, ", ".join(to), subject, body)print('Yes')


In [ ]:
smtp_server = smtplib.SMTP('smtp.gmail.com', 587)
smtp_server.ehlo()
smtp_server.starttls()
smtp_server.login('ocagnv@gmail.com', '2700SW13th')
smtp_server.sendmail('ocagnv@gmail.com', 'doug.ray@starbanner.com', 'Subject: Test from script\nHi Doug, this is your first email from a script you wrote! Cheers, me.')
smtp_server.quit()
print ('Email sent successfully')

In [ ]:
# send_email.py

from email.headerregistry import Address
from email.message import EmailMessage
import os


# Gmail details
GMAIL_ADDRESS = 'ocagnv@gmail.com' #later pull out as env var in cron script
GMAIL_APPLICATION_PASSWORD = '2700SW13th' #later pull out as env var in cron script

email_address = os.getenv('GMAIL_ADDRESS', None)
email_password = os.getenv('GMAIL_APPLICATION_PASSWORD', None)

# Recipent
to_address = (
    Address(display_name='Doug Ray', username='ocagnv', domain='gmail.com'),
)


def create_email_message(from_address, to_address, subject, body):
    msg = EmailMessage()
    msg['From'] = from_address
    msg['To'] = to_address
    msg['Subject'] = subject
    msg.set_content(body)
    return msg


if __name__ == '__main__':
    msg = create_email_message(
        from_address=email_address,
        to_address=to_address,
        subject='Hello World',
        body="Test sending the body.",
    )

    with smtplib.SMTP('smtp.gmail.com', port=587) as smtp_server:
        smtp_server.ehlo()
        smtp_server.starttls()
        smtp_server.login(email_address, email_password)
        smtp_server.send_message(msg)

    print('Email sent successfully')
    
